In [1]:
import pickle
import numpy as np
import tensorflow as tf

image_size = 28
num_labels = 10

In [2]:
class NotMNIST:
    def __init__(self):
        class Train:
            def __init__(self):
                self.images = []
                self.labels = []
                self.batch_counter = 0

            @property
            def num_examples(self):
                return len(self.images)

            def next_batch(self, num):
                if self.batch_counter + num >= len(self.labels):
                    batch_images = self.images[self.batch_counter:]
                    batch_labels = self.labels[self.batch_counter:]
                    left = num - len(batch_labels)
                    self.batch_counter = left
                else:
                    batch_images = self.images[self.batch_counter:self.batch_counter + num]
                    batch_labels = self.labels[self.batch_counter:self.batch_counter + num]
                    self.batch_counter += num
                return batch_images, batch_labels

        class Test:
            def __init__(self):
                self.images = []
                self.labels = []

        self.train = Train()
        self.test = Test()
            
        self.load_as(self.format_of((-1, image_size * image_size)))
        
    def format_of(self, shape):
        return lambda dataset, labels: (
                dataset.reshape(shape).astype(np.float32),
                (np.arange(num_labels) == labels[:, None]).astype(np.float32)
            )
        
    def load_as(self, reformat):

        pickle_file = '../data/notMNIST.pickle'

        with open(pickle_file, 'rb') as f:
            save = pickle.load(f)
            train_dataset = save['train_dataset']
            train_labels = save['train_labels']
            valid_dataset = save['valid_dataset']
            valid_labels = save['valid_labels']
            test_dataset = save['test_dataset']
            test_labels = save['test_labels']
            del save  # 删除内存文件，等待gc回收释放内存
        
        train_dataset, train_labels = reformat(train_dataset, train_labels)
        valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
        test_dataset, test_labels = reformat(test_dataset, test_labels)
        print('Training set', train_dataset.shape, train_labels.shape)
        print('Validation set', valid_dataset.shape, valid_labels.shape)
        print('Test set', test_dataset.shape, test_labels.shape)
        self.train.images = train_dataset
        self.train.labels = train_labels
        self.test.images = test_dataset
        self.test.labels = test_labels  
        
    def reshape_like_image(self, channels):
        self.load_as(self.format_of((-1, image_size, image_size, channels)))      

not_mnist = NotMNIST()

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [3]:
num_channels = 1 # 灰度图

not_mnist.reshape_like_image(num_channels)

Training set (200000, 28, 28, 1) (200000, 10)
Validation set (10000, 28, 28, 1) (10000, 10)
Test set (10000, 28, 28, 1) (10000, 10)


In [5]:
# 训练参数
learning_rate = 0.01  # 梯度下降步长，寻找最优解的下降步长
training_epochs = 25  # 迭代轮数
batch_size = 50  # 批次训练数据集大小
display_step = 1

# tf图输入
x = tf.placeholder(tf.float32, [None, image_size, image_size, num_channels])  # mnist 数据集图片大小为28*28=784，placeholder为占位符
y = tf.placeholder(tf.float32, [None, num_labels])  # 识别A-J的数据，一共10个类别

# CNN参数
patch_size = 5
depth = 16
num_hidden = 64

# Variables.
layer1_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, num_channels, 6], stddev=0.1))
layer1_biases = tf.Variable(tf.zeros([6]))

layer2_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, 6, 16], stddev=0.1))
layer2_biases = tf.Variable(tf.zeros([16]))

layer3_weights = tf.Variable(tf.truncated_normal([5 * 5 * 16, 120], stddev=0.1))
layer3_biases = tf.Variable(tf.zeros([120]))

layer4_weights = tf.Variable(tf.truncated_normal([120, 84], stddev=0.1))
layer4_biases = tf.Variable(tf.zeros([84]))

layer5_weights = tf.Variable(tf.truncated_normal([84, num_labels], stddev=0.1))
layer5_biases = tf.Variable(tf.zeros([num_labels]))
# Model.
def model(data):
    #con1
    conv = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer1_biases)
    #subpooling
    hidden = hidden = tf.nn.avg_pool(hidden, [1, 2, 2, 1], [1, 2, 2, 1], 'SAME')
    #conv2
    conv = tf.nn.conv2d(hidden, layer2_weights, [1, 1, 1, 1], padding='VALID')
    hidden = tf.nn.relu(conv + layer2_biases)
    #subpooling
    hidden = tf.nn.avg_pool(hidden, [1, 2, 2, 1], [1, 2, 2, 1], 'SAME')
    #full connect 1
    shape = hidden.get_shape().as_list()
    reshape = tf.reshape(hidden, [-1, shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    #full connect 2
    hidden = tf.nn.relu(tf.matmul(hidden, layer4_weights) + layer4_biases)
    #output
    noise = tf.random_normal(shape=tf.shape(hidden), mean=0.0, stddev=0.1, dtype=tf.float32)
    hidden = noise + hidden
    output = tf.matmul(hidden, layer5_weights) + layer5_biases
    return output
     

# 构造模型
pred = tf.nn.softmax(model(x))  # Softmax

# 损失函数：交叉墒
loss = tf.reduce_mean(-tf.reduce_sum(y * tf.log(pred), reduction_indices=1))
# 使用梯度下降算法寻找最优解
# API tf.train.GradientDescentOptimizer
# TODO 创建梯度下降优化方法
optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)
# 初始化所有变量
init = tf.global_variables_initializer()
saver = tf.train.Saver()

def accuracy(pred, y, test_data):
    # 测试模型
    correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
    # 计算准确度
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    return accuracy.eval({x: test_data.images, y: test_data.labels})

In [6]:
# 启动tf图
with tf.Session() as sess:
    sess.run(init)  # 初始化所有变量

    # 迭代训练
    for epoch in range(training_epochs):
        avg_loss = 0.
        total_batch = int(not_mnist.train.num_examples / batch_size)  # 计算数据集总共有多少批次数据
        # 训练训练所有批次数据
        for i in range(total_batch):
            batch_xs, batch_ys = not_mnist.train.next_batch(batch_size)
            # 运行优化操作和损失函数计算操作，获取损失值
            _, c = sess.run([optimizer, loss], feed_dict={x: batch_xs,
                                                          y: batch_ys})
            # 计算平均损失
            avg_loss += c / total_batch
        # 打印显示
        if (epoch + 1) % display_step == 0:
            print("Epoch:", '%04d' % (epoch + 1), "loss:", "{:.9f}".format(avg_loss),
                  "accuracy:", accuracy(pred, y, not_mnist.test))

    print("Optimization Finished!")
    print("Accuracy:", accuracy(pred, y, not_mnist.test))
    saver.save(sess, 'my-model')

Epoch: 0001 loss: 0.792829882 accuracy: 0.9037
Epoch: 0002 loss: 0.494312739 accuracy: 0.9156
Epoch: 0003 loss: 0.444530127 accuracy: 0.9246
Epoch: 0004 loss: 0.413679214 accuracy: 0.9315
Epoch: 0005 loss: 0.391518641 accuracy: 0.9372
Epoch: 0006 loss: 0.374083733 accuracy: 0.9388
Epoch: 0007 loss: 0.359843516 accuracy: 0.9422
Epoch: 0008 loss: 0.347963445 accuracy: 0.9436
Epoch: 0009 loss: 0.337919140 accuracy: 0.9461
Epoch: 0010 loss: 0.329117875 accuracy: 0.9474
Epoch: 0011 loss: 0.321322018 accuracy: 0.9482
Epoch: 0012 loss: 0.313948866 accuracy: 0.9496
Epoch: 0013 loss: 0.307476835 accuracy: 0.9521
Epoch: 0014 loss: 0.301589546 accuracy: 0.9522
Epoch: 0015 loss: 0.296422766 accuracy: 0.9539
Epoch: 0016 loss: 0.291493172 accuracy: 0.9543
Epoch: 0017 loss: 0.287127336 accuracy: 0.9538
Epoch: 0018 loss: 0.282768714 accuracy: 0.9552
Epoch: 0019 loss: 0.278883264 accuracy: 0.9552
Epoch: 0020 loss: 0.274622201 accuracy: 0.9563
Epoch: 0021 loss: 0.271465003 accuracy: 0.9551
Epoch: 0022 l